In [1]:
# Author: bbaasan
# Date: 2023-08-15
# Subject: 
# Email: bbaasan@gmu.edu

In [1]:
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import os
import glob
import re
import numpy as np
from nltk.tokenize import word_tokenize

In [2]:
pd.read_pickle('../data/Seven/seven_0005')

,tagged,transcript,text
0,"Xtant Medical Holdings, Inc. (NYSE:XTNT) Q3 20...","[Xtant Medical Holdings, Inc. (NYSE:XTNT) Q3 2...",xtant medical holdings inc nyse xtnt earnings ...
1,Uni-Select Inc. (OTCPK:UNIEF) Q3 2021 Earnings...,[Uni-Select Inc. (OTCPK:UNIEF) Q3 2021 Earning...,uni select inc otcpk unief earnings conference...
2,"Coupang, Inc. (NYSE:CPNG) Q3 2021 Earnings Con...","[Coupang, Inc. (NYSE:CPNG) Q3 2021 Earnings Co...",coupang inc nyse cpng earnings conference call...
3,Dentsu Group Inc. (OTCPK:DNTUF) Q3 2021 Earnin...,[Dentsu Group Inc. (OTCPK:DNTUF) Q3 2021 Earni...,dentsu group inc otcpk dntuf earnings conferen...
4,SGL Carbon SE (OTCPK:SGLFF) Q3 2021 Earnings C...,[SGL Carbon SE (OTCPK:SGLFF) Q3 2021 Earnings ...,sgl carbon otcpk sglff earnings conference cal...
...,...,...,...
5006,"SunOpta, Inc. (NASDAQ:STKL) Q2 2015 Earnings C...","[SunOpta, Inc. (NASDAQ:STKL) Q2 2015 Earnings ...",sunopta inc nasdaq stkl earnings conference ca...
5007,"Shell Midstream Partners, LP (NYSE:SHLX) Q2 20...","[Shell Midstream Partners, LP (NYSE:SHLX) Q2 2...",shell midstream partners nyse shlx earnings co...
5008,"Nova LifeStyle, Inc. (NASDAQ:NVFY) Q2 2015 Ear...","[Nova LifeStyle, Inc. (NASDAQ:NVFY) Q2 2015 Ea...",nova lifestyle inc nasdaq nvfy earnings confer...
5009,Forbes Energy Services Ltd. (NASDAQ:FES) Q2 20...,[Forbes Energy Services Ltd. (NASDAQ:FES) Q2 2...,forbes energy services ltd nasdaq fes results ...


In [9]:
test_0005[['tagged', 'transcript', 'text']].to_pickle('../data/Seven/seven_0005')

In [19]:
counterizer = CountVectorizer()

In [25]:
demo_0005_copy = demo_0005[demo_0005['tagged'] !='NoScript'].copy()

In [28]:
demo_0005_copy.reset_index(inplace=True, drop=True)

In [30]:
X = counterizer.fit_transform(demo_0005_copy['text'])

In [45]:
y = demo_0005_copy['target'].to_list()

In [49]:
# Assuming X_train and y_train are your training features and labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the classifier
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

# Predict on validation data
y_pred = rf_classifier.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)


Validation Accuracy: 0.5811240721102863


In [52]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.58      0.98      0.73       542
           1       0.61      0.04      0.08       401

    accuracy                           0.58       943
   macro avg       0.59      0.51      0.40       943
weighted avg       0.59      0.58      0.45       943



## Parameter Analysis  

- change of n_estimator  

- change of test size  

In [94]:
# Assuming X_train and y_train are your training features and labels

list_nestimators = [50, 100, 150, 200]

for n_estimator in list_nestimators:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize the classifier
    rf_classifier = RandomForestClassifier(n_estimators=n_estimator, max_depth=10, random_state=42)

    # Train the model
    rf_classifier.fit(X_train, y_train)

    # Predict on validation data
    y_pred = rf_classifier.predict(X_test)

    report = classification_report(y_test, y_pred)
    print(f'\nprinting on n_estimator: {n_estimator}\n')
    print(report)


printing on n_estimator: 50

              precision    recall  f1-score   support

           0       0.59      0.97      0.73       542
           1       0.64      0.07      0.13       401

    accuracy                           0.59       943
   macro avg       0.61      0.52      0.43       943
weighted avg       0.61      0.59      0.47       943


printing on n_estimator: 100

              precision    recall  f1-score   support

           0       0.58      0.98      0.73       542
           1       0.61      0.04      0.08       401

    accuracy                           0.58       943
   macro avg       0.59      0.51      0.40       943
weighted avg       0.59      0.58      0.45       943


printing on n_estimator: 150

              precision    recall  f1-score   support

           0       0.58      0.99      0.73       542
           1       0.60      0.03      0.06       401

    accuracy                           0.58       943
   macro avg       0.59      0.51   

## Cross Validation  

- 5-fold-cross-validation  



In [64]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initializing the classifier
rf_classifier = RandomForestClassifier(n_estimators=100)

# Lists to store accuracy scores from each fold
accuracy_scores = []


# K-fold cross-validation loop
for fold_number, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]
    
    # Train the model on the training data
    rf_classifier.fit(X_train, y_train)
    
    # Make predictions on the validation data
    y_pred = rf_classifier.predict(X_test)
    
    # Calculate accuracy for this fold
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

    report = classification_report(y_test, y_pred)

    print(f'Classification Report - Fold: {fold_number}')
    print(f'\n{report}\n\n')


# Calculate the average accuracy across all folds
#average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
#print("Average Accuracy:", average_accuracy)


Classification Report - Fold: 0

              precision    recall  f1-score   support

           0       0.59      0.92      0.72       542
           1       0.59      0.15      0.24       401

    accuracy                           0.59       943
   macro avg       0.59      0.54      0.48       943
weighted avg       0.59      0.59      0.52       943



Classification Report - Fold: 1

              precision    recall  f1-score   support

           0       0.63      0.92      0.74       568
           1       0.57      0.17      0.26       375

    accuracy                           0.62       943
   macro avg       0.60      0.54      0.50       943
weighted avg       0.60      0.62      0.55       943



Classification Report - Fold: 2

              precision    recall  f1-score   support

           0       0.64      0.91      0.75       578
           1       0.57      0.18      0.28       365

    accuracy                           0.63       943
   macro avg       0.60  

In [73]:
midsize = pd.concat([demo_0005,demo_0015,demo_0025], axis=0, ignore_index=True)
midsize.columns

Index(['id', 'url', 'transcript', 'recorded_date', 'date', 'tagged',
       'stock_ticker', 'stock', 'tick', 'meeting_date', 'close', 'volume',
       'sma50_close', 'sma200_close', 'sma50_vol', 'sma200_vol', 'clean',
       'text', 'target'],
      dtype='object')

In [85]:
midsize_copy = midsize[midsize['tagged'] != 'NoScript']


In [88]:
vectorizer_mid = CountVectorizer()

X_mid = vectorizer_mid.fit_transform(midsize_copy['text'])

In [90]:
y_mid = midsize_copy['target'].to_list()

In [92]:
X_mid_train, X_mid_test, y_mid_train, y_mid_test = train_test_split(X_mid, y_mid, test_size=0.2, random_state=42)

# Initialize the classifier
rf_classifier_mid = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# Train the model
rf_classifier_mid.fit(X_mid_train, y_mid_train)

# Predict on validation data
y_mid_pred = rf_classifier_mid.predict(X_mid_test)

report_mid = classification_report(y_mid_test, y_mid_pred)
print(report_mid)


              precision    recall  f1-score   support

           0       0.60      0.99      0.75      2844
           1       0.64      0.02      0.03      1887

    accuracy                           0.60      4731
   macro avg       0.62      0.50      0.39      4731
weighted avg       0.62      0.60      0.46      4731



In [93]:
cm_mid = confusion_matrix(y_mid_test, y_mid_pred)
cm_mid

array([[2828,   16],
       [1858,   29]], dtype=int64)

In [96]:
list_nestimators = [50, 100, 150, 200]

for n_estimator in list_nestimators:
    X_mid_train, X_mid_test, y_mid_train, y_mid_test = train_test_split(X_mid, y_mid, test_size=0.2, random_state=42)

    # Initialize the classifier
    rf_classifier_mid = RandomForestClassifier(n_estimators=n_estimator, max_depth=10, random_state=42)

    # Train the model
    rf_classifier_mid.fit(X_mid_train, y_mid_train)

    # Predict on validation data
    y_mid_pred = rf_classifier_mid.predict(X_mid_test)

    report_mid = classification_report(y_mid_test, y_mid_pred)
    print(f'\nprinting on n_estimator: {n_estimator}\n')
    print(report_mid, '\n')

    cm_mid = confusion_matrix(y_mid_test, y_mid_pred)
    print(f'confusion matrix for {n_estimator}\n', cm_mid)


printing on n_estimator: 50

              precision    recall  f1-score   support

           0       0.60      0.99      0.75      2844
           1       0.61      0.02      0.05      1887

    accuracy                           0.60      4731
   macro avg       0.61      0.51      0.40      4731
weighted avg       0.61      0.60      0.47      4731
 

confusion matrix for 50
 [[2815   29]
 [1841   46]]

printing on n_estimator: 100

              precision    recall  f1-score   support

           0       0.60      0.99      0.75      2844
           1       0.64      0.02      0.03      1887

    accuracy                           0.60      4731
   macro avg       0.62      0.50      0.39      4731
weighted avg       0.62      0.60      0.46      4731
 

confusion matrix for 100
 [[2828   16]
 [1858   29]]

printing on n_estimator: 150

              precision    recall  f1-score   support

           0       0.60      1.00      0.75      2844
           1       0.68      0.01   

In [133]:
def report_to_dataframe(report_str=None, report_name: str=None):
    
    with open(f'../data/results/{report_name}.txt', mode='w') as txt_file:
        txt_file.write(report_str)

In [134]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_mid_test, y_mid_pred)
mse

0.39611075882477276

In [135]:
print(confusion_matrix(y_mid_test, y_mid_pred))

[[2834   10]
 [1864   23]]


In [137]:
midsize_copy.target.value_counts()

target
0    14046
1     9606
Name: count, dtype: int64

#### Addressing overfitting and imbalance of category with PCA

In [139]:
from sklearn.decomposition import PCA
test_0005['text']

0       xtant medical holdings inc nyse xtnt earnings ...
1       uni select inc otcpk unief earnings conference...
2       coupang inc nyse cpng earnings conference call...
3       dentsu group inc otcpk dntuf earnings conferen...
4       sgl carbon otcpk sglff earnings conference cal...
                              ...                        
5006    sunopta inc nasdaq stkl earnings conference ca...
5007    shell midstream partners nyse shlx earnings co...
5008    nova lifestyle inc nasdaq nvfy earnings confer...
5009    forbes energy services ltd nasdaq fes results ...
5010    intercloud systems inc otc icld earnings confe...
Name: text, Length: 5011, dtype: object

In [141]:
from sklearn.decomposition import TruncatedSVD

tfidf_vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(test_0005['text'])

# Apply Truncated SVD for dimensionality reduction
num_components = 100  # Specify the number of components to retain
svd = TruncatedSVD(n_components=num_components)
reduced_matrix = svd.fit_transform(tfidf_matrix)

print(pd.DataFrame(reduced_matrix, columns=[svd.get_feature_names_out()]))



def pca_example():
    from sklearn.datasets import fetch_20newsgroups
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.decomposition import TruncatedSVD

    # Load example text data
    newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
    data = newsgroups.data

    # Convert text data to TF-IDF matrix
    tfidf_vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(data)

    # Apply Truncated SVD for dimensionality reduction
    num_components = 100  # Specify the number of components to retain
    svd = TruncatedSVD(n_components=num_components)
    reduced_matrix = svd.fit_transform(tfidf_matrix)

    print("Original matrix shape:", tfidf_matrix.shape)
    print("Reduced matrix shape:", reduced_matrix.shape)



     truncatedsvd0 truncatedsvd1 truncatedsvd2 truncatedsvd3 truncatedsvd4  \
0         0.400878      0.019089     -0.013690      0.216317     -0.072774   
1         0.357825      0.006239     -0.036473      0.024958      0.011891   
2         0.445793      0.008105     -0.019657     -0.061863     -0.131750   
3         0.372915      0.000030     -0.028121     -0.027063     -0.129369   
4         0.475347     -0.009565     -0.045160     -0.011777      0.010355   
...            ...           ...           ...           ...           ...   
5006      0.443525     -0.005085     -0.036774      0.024023     -0.014332   
5007      0.311711      0.002888     -0.007620     -0.031687      0.083429   
5008      0.371955      0.002351     -0.013087      0.140278     -0.052851   
5009      0.332653     -0.003006     -0.001766     -0.107470      0.064582   
5010      0.394306     -0.006403     -0.035697      0.163217     -0.136101   

     truncatedsvd5 truncatedsvd6 truncatedsvd7 truncatedsvd8 tr

In [167]:
from nltk.corpus import stopwords
my_stopwords = pd.read_pickle("E:\\AI\\AI\\stopwords.pkl")['stopwords'].to_list()
my_stopwords.extend(stopwords.words('english'))

new_list = ['think','year','million', 'question', 'going', 'see', 'well','also','call',
            'thank', 'new', 'like','one','good','really', 'operator', 'continue','next',
            'second','time','second','kind','today','get','right','line','fourth', 'strong',
            'end', 'little', 'get','thanks','know','terms', 'could','okay','third','say','two',
            'maybe','great','years','still','number','may','year','lot','back','basis','morning',
            'conference','things','approximately','total','turn','questions','mon', 'bit','make',
            'work','part','based','billion','term','half','john','full','mentioned','long','sort',
            'high','due','prior']

my_stopwords.extend(new_list)

# Create TF-IDF vectors
vectorizer = TfidfVectorizer(max_features=10000, stop_words=my_stopwords)
tfidf_matrix = vectorizer.fit_transform(test_0005['text'])

# Apply Truncated SVD
num_components = 100
svd = TruncatedSVD(n_components=num_components)
X_reduced = svd.fit_transform(tfidf_matrix)

components_df = pd.DataFrame(X_reduced, columns=[f'Component_{i}' for i in range(num_components)])
print(components_df)


      Component_0  Component_1  Component_2  Component_3  Component_4  \
0        0.418587     0.005043     0.010521    -0.095513     0.038239   
1        0.261141     0.000485    -0.032881    -0.005924    -0.020699   
2        0.371048     0.003316    -0.027191    -0.116766    -0.027208   
3        0.210736    -0.001110    -0.019243    -0.068700     0.004764   
4        0.343336    -0.012340    -0.033874     0.011411    -0.225008   
5        0.171973     0.002813    -0.012030     0.040464    -0.089061   
6        0.258876    -0.010125    -0.018104     0.009207    -0.019029   
7        0.281999    -0.009125     0.020189     0.013356     0.046113   
8        0.370372    -0.012513    -0.050701    -0.039220    -0.038084   
9        0.400802    -0.013471    -0.056361     0.005473    -0.137960   
10       0.315024    -0.002429    -0.011907     0.045493    -0.128615   
11       0.401323    -0.013372    -0.020046    -0.112541     0.042767   
12       0.340453    -0.005000    -0.012585    -0.0

In [168]:
components_vocab_df = pd.DataFrame(svd.components_, columns=vectorizer.get_feature_names_out())


In [170]:
pd.reset_option('display.max_rows')
components_vocab_df

,aam,aaron,abate,abbott,abbvie,abilities,ability,abl,abnormal,abrahams,...,zhou,zillow,zimmer,zimmerman,zinc,zip,zipcar,zone,zones,zoom
0,0.001027,0.008511,0.001168,0.001640,0.000854,0.000986,0.024487,0.002031,0.000917,0.000653,...,0.001536,0.001119,0.001100,0.001345,0.002686,0.000866,0.001183,0.003581,0.001894,0.001296
1,-0.000073,-0.000350,-0.000058,-0.000053,-0.000041,-0.000048,-0.001095,-0.000104,-0.000027,-0.000026,...,0.000069,-0.000062,-0.000042,-0.000067,-0.000108,-0.000026,-0.000071,-0.000103,-0.000019,-0.000061
2,-0.000520,-0.001359,-0.000852,0.009043,0.009654,-0.000064,0.004537,-0.001445,0.001000,0.005892,...,-0.000476,-0.000106,-0.000282,0.002228,0.000004,-0.000027,-0.001030,0.000038,-0.000432,0.000058
3,-0.000737,-0.001917,0.000674,-0.000965,0.001189,-0.000956,-0.008014,0.001257,0.000342,0.001916,...,-0.004947,-0.003838,0.002175,0.000123,0.013823,-0.000251,0.000176,0.011161,0.008065,-0.003126
4,-0.002599,0.004944,0.000867,0.001047,0.000710,-0.000427,-0.001115,-0.000040,-0.000686,0.000868,...,0.002394,0.002965,0.002228,0.000899,-0.022633,0.000304,-0.001932,-0.017277,-0.013336,0.000067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.005194,-0.026673,0.000628,-0.004892,0.002423,-0.001534,0.005185,0.001083,-0.000344,-0.000007,...,0.001982,0.021792,-0.015133,-0.020100,-0.018202,0.004271,0.022894,-0.002277,0.000608,0.008158
96,-0.003234,-0.030723,-0.001477,-0.000520,-0.006522,-0.001212,0.002004,0.002013,0.000341,-0.001655,...,-0.007076,0.008764,-0.012019,0.001078,0.003917,0.003980,0.019206,-0.001894,0.001950,-0.003243
97,0.001846,-0.065649,0.004210,-0.008139,-0.013150,-0.000388,-0.005602,0.004412,-0.000583,-0.006758,...,0.000572,-0.019718,-0.015556,0.006845,-0.031026,-0.001202,0.020236,0.005251,-0.004461,0.003170
98,0.002744,-0.018153,0.000956,-0.005279,-0.001099,0.001897,0.002178,0.004018,-0.000317,0.001695,...,0.001008,0.072817,0.013268,0.003418,-0.008583,0.003938,0.003745,0.000119,-0.003057,-0.004827


In [ ]:
# Assuming X_train and y_train are your training features and labels
X_reduced_train, X_reduced_test, y_reduced_train, y_reduced_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

# Initialize the classifier
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

# Predict on validation data
y_pred = rf_classifier.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)